In [ ]:
from __future__ import print_function, division

import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
from get_metrics import *
plt.ion()  

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

In [ ]:
ROOT = 'mura_clahe'
data_dir = os.path.join(ROOT, 'mura')
images_dir = os.path.join(data_dir, 'images')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

In [ ]:
pretrained_size = 320

pretrained_means = [0.485,0.456,0.406]

pretrained_stds= [0.229,0.224,0.225]

batch_size=8

data_transforms = {
    
    'train': transforms.Compose([
        transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomRotation(10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
    ]),
    'val': transforms.Compose([
        transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
    ]),
                   }
print("Initializing Datasets and Dataloaders...\n")
# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True) for x in ['train', 'val']}
device = torch.device("cuda:0")
dataset_sizes ={x:len(image_datasets[x]) for x in ['train','val']}

In [ ]:
# Class names convert to index
image_datasets['train'].class_to_idx
class_names=image_datasets['train'].classes
print(">>Class Names: {}\n".format(image_datasets['train'].classes))
print(">>Class Index: {}\n".format(image_datasets['train'].class_to_idx))
print(">>Number of images in training={}\n".format(dataset_sizes['train']))
print(">>Number of images in test={}\n".format(dataset_sizes['val']))
print("    Number of steps for training set={}\n".format(len(dataloaders['train'])))
print("    Number of steps for test set={}\n".format(len(dataloaders['val']))) 
# 1:positive #0:negative

In [ ]:
train_losses = []
train_acc = []
val_losses = []
val_acc = []

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    ghost=True
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs,aux= model(inputs) only for ınception network
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.detach() * inputs.size(0) #.item()
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/ dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            #Losses and accuracy per epochs are stored in array for plot graphs
            if phase == 'train':              
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'val':
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)
            torch.cuda.empty_cache()

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        torch.cuda.empty_cache()
    # Determine total traning time
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))    
    # Print best validation accuracy 
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(best_model_wts, "./model.pth")
    return model

In [ ]:
#Resnet Series
model_ft=models.resnet34(pretrained=True)#models.resnet34,models.resnet50,models.resnet101,models.resnet152,
num_ftrs = model_ft.fc.in_features
model_ft.fc =nn.Sequential(nn.Linear(num_ftrs, 2),
nn.Sigmoid())

In [ ]:
"""
#Resnext Series
model_ft =models.resnext50_32x4d(pretrained=True)#models.resnext50_32x4d,models.resnext101_32x8d
num_ftrs = model_ft.fc.in_features
model_ft.fc =nn.Sequential(nn.Linear(num_ftrs, 2),
nn.Sigmoid())
"""

In [ ]:
"""
#VGG Series
model_ft =models.vgg13(pretrained=True)#models.vgg13,models.vgg16,models.vgg19
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Sequential(nn.Linear(num_ftrs, 2))
"""

In [ ]:
"""
#Densenet Series
model_ft =models.densenet169(pretrained=True)#models.densenet169,models.densenet201
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, 2),
nn.Sigmoid())
"""

In [ ]:
"""
#Inception v3 
model_ft =models.inception_v3(pretrained=True)
model_ft.AuxLogits.fc = nn.Sequential(nn.Linear(768, 2),
nn.Sigmoid())
model_ft.fc = nn.Sequential(nn.Linear(2048, 2),
nn.Sigmoid())
"""

In [ ]:
"""
#Mobilenet v2
model_ft =models.mobilenet_v2(pretrained=True)
num_ftrs=model_ft.last_channel
model_ft.classifier[1] = nn.Sequential(nn.Linear(num_ftrs, 2),
nn.Sigmoid())
"""

In [ ]:
#model_ft.load_state_dict(torch.load("./model.pth"))

In [ ]:
# Model send to GPU
model_ft = model_ft.to(device)

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model_ft.parameters(), betas=(0.9,0.999), lr=0.0001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer, exp_lr_scheduler,40)

# Graphs

In [ ]:
#This code block draw graph for accuracy
matplotlib.rcdefaults()
epochs=range(0, 40, 1)
plt.plot(epochs, train_acc, color='#006BA4')
plt.plot(epochs, val_acc, color='#FF800E')
plt.grid(b=True, which='major', color='lightgray')
plt.grid(b=True, which='minor', color='lightgray')
plt.xticks(np.arange(0, 45, 5))
plt.yticks(np.arange(0.5, 1, 0.05))
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 600
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['Training Acc.', 'Validation Acc.'],loc='lower right')

In [ ]:
#This code block draw graph for loss
matplotlib.rcdefaults()
plt.plot(epochs, train_losses, color='#006BA4')
plt.plot(epochs, val_losses, color='#FF800E')
plt.grid(b=True, which='major', color='lightgray')
plt.grid(b=True, which='minor', color='lightgray')
plt.xticks(np.arange(0, 45, 5))
plt.yticks(np.arange(0, 1.2, 0.2))
plt.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['Training Loss', 'Validation Loss'],loc='lower right')

 #                                                Model Test

In [ ]:
get_metric.test_model(model_ft,device,dataloaders['val'])

In [ ]:
actuals, predictions = get_metric.test_label_predictions(model_ft, device, dataloaders['val'])

In [ ]:
get_metric.get_classification_report(actuals, predictions)

In [ ]:
get_metric.get_confusion_matrix(actuals, predictions)

In [ ]:
get_metric.get_cohen_kappa(actuals, predictions)

In [ ]:
get_metric.get_roc_curves(model_ft, device,  dataloaders['val'])